<a href="https://colab.research.google.com/github/g-lam/colab-samples/blob/main/Text-to-Image/Stable_Diffusion_XL_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate invisible_watermark mediapy

In [ ]:
use_refiner = True

In [ ]:
import mediapy as media
import random
import sys
import time
import torch

from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    )

if use_refiner:
  refiner = DiffusionPipeline.from_pretrained(
      "stabilityai/stable-diffusion-xl-refiner-1.0",
      text_encoder_2=pipe.text_encoder_2,
      vae=pipe.vae,
      torch_dtype=torch.float16,
      use_safetensors=True,
      variant="fp16",
  )

  refiner = refiner.to("cuda")

  pipe.enable_model_cpu_offload()
else:
  pipe = pipe.to("cuda")

In [ ]:
prompt = "Charlie Brown and Lucy sitting down for a meal at a fine dining restaurant with a view to the Eiffel Tower, Professional Photography"
seed = random.randint(0, sys.maxsize)

negative_prompt = "cartoon, anime, (deformed eyes, nose, ears, body, limbs, fingers), bad anatomy, ugly"

start = time.time()
images = pipe(
    prompt = prompt,
    negative_prompt = negative_prompt,
    output_type = "latent" if use_refiner else "pil",
    generator = torch.Generator("cuda").manual_seed(seed),
    ).images

if use_refiner:
  images = refiner(
      prompt = prompt,
      negative_prompt = negative_prompt,
      image = images,
      ).images
end = time.time()

print(f"Prompt:\t{prompt}\nSeed:\t{seed}\nTime:\t{(end - start)}s")
media.show_images(images)
images[0].save(f"output-{str(int(time.time()))[-6:]}.jpg")